In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import requests
import codecs
import _pickle as pickle
import time

In [4]:
def read_conll_format(input_file):
    with codecs.open(input_file, 'r', encoding='utf8') as f:
        word_list = [] 
        chunk_list = []
        pos_list = []
        tag_list = []
        words = []
        chunks = []
        poss = []
        tags = []
        num_sent = 0
        max_length = 0
        for line in f:
            line = line.split()
            if len(line) > 0:
                words.append(map_number_and_punct(line[0].lower()))
                poss.append(line[1])
                chunks.append(line[2])
                tags.append(line[3])
            else:
                word_list.append(words)
                pos_list.append(poss)
                chunk_list.append(chunks)
                tag_list.append(tags)
                sent_length = len(words)
                words = []
                chunks = []
                poss = []
                tags = []
                num_sent += 1
                max_length = max(max_length, sent_length)
    return word_list, pos_list, chunk_list, tag_list, num_sent, max_length

In [5]:
def map_number_and_punct(word):
    if any(char.isdigit() for char in word):
        word = u'<number>'
    elif word in [u',', u'<', u'.', u'>', u'/', u'?', u'..', u'...', u'....', u':', u';', u'"', u"'", u'[', u'{', u']',
                  u'}', u'|', u'\\', u'`', u'~', u'!', u'@', u'#', u'$', u'%', u'^', u'&', u'*', u'(', u')', u'-', u'+',
                  u'=']:
        word = u'<punct>'
    return word

In [13]:
word_list, pos_list, chunk_list, tag_list, num_sent, max_length = read_conll_format('/content/drive/MyDrive/data/data_cau_giay/all_error_spelling_cover.txt')

In [14]:
def dict_of_tags(tag_list):
    dic = {'pad': 0}
    index = 0
    for tags in tag_list:
        for tag in tags:
            try:
                dic[tag]
            except:
                index = index + 1
                dic[tag]= index
    return dic, len(dic)

In [15]:
# print(tag_list)
dic_of_tag, len_of_dic_tag = dict_of_tags(tag_list)
print(dic_of_tag)
print(len_of_dic_tag)

{'pad': 0, 'đường': 1, 'trần': 2, 'thái': 3, 'tông': 4, 'trung': 5, 'hoà': 6, 'cầu': 7, 'giấy': 8, 'hà': 9, 'nội': 10, 'ngõ': 11, '195': 12, 'kính': 13, '63': 14, 'xuân': 15, 'thủy': 16, 'nguyễn': 17, 'chánh': 18, 'ngách': 19, '43/121': 20, 'phố': 21, 'quốc': 22, 'hoàn': 23, '43/113': 24, '2': 25, '53': 26, '41': 27, 'dương': 28, 'đình': 29, 'nghệ': 30, '53/15': 31, '23': 32, '85': 33, '106/43': 34, 'hẻm': 35, '181/27/32': 36, '20': 37, '37': 38, '31': 39, 'mạc': 40, '31/26': 41, '181/27/32/18': 42, '123': 43, '31/22': 44, '20/5': 45, '181/27': 46, '193': 47, 'yên': 48, '31/18': 49, '181/27/10': 50, '48': 51, '31/12': 52, '85/24/10': 53, 'hoàng': 54, 'minh': 55, 'giám': 56, 'doãn': 57, 'kế': 58, 'thiện': 59, 'mai': 60, 'dịch': 61, '445': 62, 'lạc': 63, 'long': 64, 'quân': 65, 'nghĩa': 66, 'đô': 67, 'giao': 68, 'lộ': 69, 'đoạn': 70, 'quay': 71, 'đầu': 72, 'võ': 73, 'chí': 74, 'công': 75, 'ngọc': 76, 'vũ': 77, 'thị': 78, 'định': 79, 'lê': 80, 'văn': 81, 'lương': 82, 'ngân': 83, '10': 84,

In [16]:
#===========================encode tag_list by dic_of_tags================
def encode_tag_list_by_dic_of_tags(tag_list, dic, len_of_a_sentence):
    encodeTag_of_many_sentence = []
    for tags_of_a_sentence in tag_list:
        # print(tags_of_a_sentence)
        encode_of_a_sentence = np.zeros(len_of_a_sentence)
        for i in range(len(tags_of_a_sentence)):
            tag = tags_of_a_sentence[i]
            encode_of_a_sentence[i] = dic[tag]
        encodeTag_of_many_sentence.append(encode_of_a_sentence)
    return np.array(encodeTag_of_many_sentence)

In [17]:
tag_list_encode = encode_tag_list_by_dic_of_tags(tag_list,dic_of_tag, 25)
print(tag_list_encode.shape)

(6929, 25)


In [19]:
def create_onehot(sens_encodes, tag_dim, num_word_in_sentence):
    X = np.zeros([len(sens_encodes), num_word_in_sentence, tag_dim])
    for i in range(len(sens_encodes)):# duyet tung cau
        for j in range(len(sens_encodes[i])):# duyet tung tu trong cau
            index = sens_encodes[i][j].astype(np.int64)
            X[i, j, index] = 1

    X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
    fileout = open('/content/drive/MyDrive/data/tag_encode_input/all_error_spelling_cover_tag_embedd.txt', 'a+', encoding='utf8')
    np.savetxt(fileout, X)
    fileout.close()
    return X

In [20]:
tag_onehot = create_onehot(tag_list_encode, tag_dim = len_of_dic_tag, num_word_in_sentence = 25)
print(tag_onehot.shape)
# print(tag_onehot[0])

(173225, 561)
